In [3]:
import os
em='bb189'
print("Joined")       
if not os.path.exists(f'../../../datadir/Gauge_Timeslices/2060_2081/{em}/'):
    os.makedirs(f'../../../datadir/Gauge_Timeslices/2060_2081/{em}/')

Joined
